In [21]:
import torch
import torchvision
import torchvision.transforms as transforms

In [22]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from datetime import datetime

In [23]:
def normalize_column(df, column_name):
    low = df_raw[column_name].min()
    high = df_raw[column_name].max()
    span = high-low
    return (df[column_name]-low)/span

def item_onehot(item, categories, length):
    idx = [*categories].index(item)
    encoding = np.zeros(length)
    encoding[idx] = 1
    return [*encoding]

def encode_onehot(col):
    categories = col.unique()
    length = len(categories)
    out_col = col.apply(lambda x: item_onehot(x, categories, length))
    return out_col, categories, length

In [24]:
df_raw = pd.read_csv("./data.csv")
df = df_raw
df['sepal_length'] = normalize_column(df_raw, 'sepal_length')
df['sepal_width'] = normalize_column(df_raw, 'sepal_width')
df['petal_length'] = normalize_column(df_raw, 'petal_length')
df['petal_width'] = normalize_column(df_raw, 'petal_width')
cats = [*df['species'].unique()]
onehot, categories, length = encode_onehot(df['species'])
df['species'] = onehot

x_cols = ["sepal_length","sepal_width","petal_length","petal_width"]

x = df[x_cols].to_numpy(dtype=float)
y = np.asarray([*df["species"].to_numpy()])

x = torch.FloatTensor(x)
y = torch.FloatTensor(y)

train = df.sample(frac=0.7)
_test = df.drop(train.index).sample(frac=1)
test = _test.sample(frac=0.7)
val = _test.drop(test.index).sample(frac=1)

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(4,5)
        self.l2 = nn.Linear(5,10)
        self.l3 = nn.Linear(10,3)
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = nn.functional.softmax(self.l3(x), dim=0)
        return x
    
net = Net()
print(net)

Net(
  (l1): Linear(in_features=4, out_features=5, bias=True)
  (l2): Linear(in_features=5, out_features=10, bias=True)
  (l3): Linear(in_features=10, out_features=3, bias=True)
)


In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [35]:
torch.device('mps')

epochs = 1000
losses = []

for i in range(epochs):
    
    old_time = datetime.now()
    
    x_train = torch.FloatTensor(train[x_cols].to_numpy())
    y_train = torch.FloatTensor(np.asarray([*train['species'].to_numpy()]))
    
    y_pred = net.forward(x_train)
    loss = criterion(y_pred, y_train)
    
    if (i % (epochs/10) == 0):
        with torch.no_grad():
            x_test = torch.FloatTensor(test[x_cols].to_numpy())
            y_test = torch.FloatTensor(np.asarray([*test['species'].to_numpy()]))
            
            y_test_pred = net(x_test)
            test_loss = criterion(y_test, y_test_pred)
            
            delta = (datetime.now() - old_time)*1000
            old_time = datetime.now()
            
            print(f'EPOCH: {i} ==> LOSS: {test_loss}')
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print("--- TRAINING FINISHED ---")

EPOCH: 0 ==> LOSS: 0.05336562916636467
EPOCH: 100 ==> LOSS: 0.05336562171578407
EPOCH: 200 ==> LOSS: 0.053365617990493774
EPOCH: 300 ==> LOSS: 0.053365617990493774
EPOCH: 400 ==> LOSS: 0.053365614265203476
EPOCH: 500 ==> LOSS: 0.05336562171578407
EPOCH: 600 ==> LOSS: 0.05336562171578407
EPOCH: 700 ==> LOSS: 0.053365614265203476
EPOCH: 800 ==> LOSS: 0.053365617990493774
EPOCH: 900 ==> LOSS: 0.053365617990493774
--- TRAINING FINISHED ---


In [47]:
idx = 10

x_val = torch.FloatTensor(val[x_cols].to_numpy())
y_val = torch.FloatTensor([*val['species'].to_numpy()])

with torch.no_grad():
    y_val_pred = net(x_val)
    val_loss = criterion(y_val, y_val_pred)
    print(f"Predicted {[*y_val_pred[idx].numpy()]}")
    print(f'Actual {[*y_val[idx].numpy()]}')
    print(f'Validation Loss {val_loss.numpy()}')

Predicted [4.575062e-09, 0.24937737, 2.1725408e-20]
Actual [0.0, 1.0, 0.0]
Validation Loss 0.1181672066450119
